# Demo of using `pymhlib` within a Jupyter notebook

Note that this notebook is still rather crude and not yet cleaned; it is a very basic first try.

In [ ]:
import sys
sys.path.append('../..')  # path to pymhlib to be used
%load_ext autoreload
%aimport pymhlib.demos.vertex_cover, logging
%autoreload 1
from pymhlib.settings import settings, parse_settings
from pymhlib.log import init_logger
from pymhlib.demos.vertex_cover import VertexCoverInstance, VertexCoverSolution
from pymhlib.gvns import GVNS, Method
import logging
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
%matplotlib inline
from IPython.display import set_matplotlib_formats; set_matplotlib_formats('png', 'pdf')
if not settings.__dict__: parse_settings(args='')
init_logger()
logger = logging.getLogger("pymhlib")

## In the following different construction heuristics, local search and a GVNS are applied 'runs' times to the vertex cover problem and box plots of the obtained solution values are created

In [ ]:
runs=3
problem_name = "gnm-1000-2000"
settings.mh_titer=10000
logger.info(f"pymhlib demo for solving {problem_name}")
# logger.info(get_settings_as_str())
res_approx = list()
res_approx_red = list()
res_deg_greedy = list()
res_deg_greedy_ls = list()
for run in range(runs):
    instance = VertexCoverInstance(problem_name+f'-{run+1}')
    # logger.info(f"{problem_name} instance read:\n" + str(instance))
    solution = VertexCoverSolution(instance)
    solution.two_approximation_construction()
    res_approx.append(solution.obj())
    solution.remove_redundant()
    res_approx_red.append(solution.obj())
    solution.greedy_construction()
    res_deg_greedy.append(solution.obj())
    alg = GVNS(solution,
            [Method(f"ch0", VertexCoverSolution.construct, 0)],
            [Method(f"li2", VertexCoverSolution.local_improve, 2)],
            [Method(f"sh{i}", VertexCoverSolution.shaking, i) for i in range(0)])
    alg.run()
    alg.method_statistics()
    alg.main_results()
    res_deg_greedy_ls.append(solution.obj())

In [ ]:
res_vns = []
settings.mh_titer=1000
for run in range(runs):
    instance = VertexCoverInstance(problem_name+f'-{run+1}')
    # logger.info(f"{problem_name} instance read:\n" + str(instance))
    solution = VertexCoverSolution(instance)
    alg = GVNS(solution,
            [Method(f"ch0", VertexCoverSolution.construct, 0)],
            [Method(f"li2", VertexCoverSolution.local_improve, 2)],
            [Method(f"sh{i}", VertexCoverSolution.shaking, i) for i in range(1,5)])
    alg.run()
    alg.method_statistics()
    alg.main_results()
    res_vns.append(solution.obj())

In [ ]:
df = pd.DataFrame({'2-apx': res_approx, '2-apx+LS1': res_approx_red, 
                   'deg-greedy': res_deg_greedy, 'deg-greedy+LS2': res_deg_greedy_ls,
                   'VNS': res_vns})
sns.boxplot(data=df[['2-apx', '2-apx+LS1', 'deg-greedy', 'deg-greedy+LS2']])
plt.title(f'Minimum Vertex Cover Problem\n{runs} random graphs with $n=1000,\,m=2000$')
plt.ylabel('$|C|$');
# plt.ylim((0,1000))
# plt.savefig("vcp1.pdf")

In [ ]:
plt.title(f'Minimum Vertex Cover Problem\n{runs} random graphs with $n=1000,\,m=2000$')
plt.ylabel('$|C|$')
sns.boxplot(data=df[['2-apx+LS1', 'deg-greedy', 'deg-greedy+LS2', 'VNS']]);
# plt.savefig("vcp2.pdf")

## The following illustrates an interactive interface to start the GVNS and create a plot of the obtained solutions values over the iterations

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
iter_text = widgets.FloatText(description="Iterations", min=0, max=10000, value=1000)
iter_slider = widgets.IntSlider(min=0, max=10000)
start = widgets.Button(description="Start")
output = widgets.Output()
hbox = widgets.HBox([iter_text, iter_slider])
display(hbox)
widgets.jslink((iter_text, 'value'), (iter_slider, 'value'))
display(start,output)

problem_name = "gnm-1000-2000"
df = None

def run(_start):
    with output:
        global logger, instance, solution, alg
        output.clear_output()
        settings.mh_log = "run.log"  # write iteration log to file
        settings.mh_lfreq = 1  # log all iterations
        init_logger()
        logger.handlers = []  # switch off general textual log output
        logger.info(f"pymhlib demo for solving {problem_name}")
        settings.mh_titer = iter_text.value
        instance = VertexCoverInstance(problem_name+f'-1')
        solution = VertexCoverSolution(instance)
        alg = GVNS(solution,
                [Method(f"ch0", VertexCoverSolution.construct, 2)],
                [Method(f"li2", VertexCoverSolution.local_improve, 2)],
                [Method(f"sh{i}", VertexCoverSolution.shaking, i) for i in range(1,5)])
        alg.run()
        # alg.method_statistics()
        # alg.main_results()
        
        logging.getLogger("pymhlib_iter").handlers[0].flush()
        global df
        df = pd.read_csv("run.log", sep=r'\s+')
        df.plot(kind='line', x='iteration', y=['best', 'obj_new']); plt.ylabel('obj')
        # display(df)
        show_inline_matplotlib_plots()
    
start.on_click(run)